An implementation of sequence to sequence learning for performing addition
Input: "535+61"
Output: "596"
Padding is handled by using a repeated sentinel character (space)
Input may optionally be reversed, shown to increase performance in many tasks in:
"Learning to Execute"
http://arxiv.org/abs/1410.4615
and
"Sequence to Sequence Learning with Neural Networks"
http://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf
Theoretically it introduces shorter term dependencies between source and target.

Modified to train RNN to conjugate past tense of verbs. Data currently taken from UCLA dataset on verb conjugation.

Code taken, modified from https://github.com/keras-team/keras/blob/master/examples/addition_rnn.py

In [1]:
from __future__ import print_function
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range
import csv
from keras.regularizers import L1L2
from matplotlib import pyplot

/Users/etang/anaconda3/envs/nlu/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Character Table

Taking in our string of numbers/operations as a string of characters, we must vectorize each character as a new input to our RNN. 

We first create a table mapping each character in our vocabulary (in our case, this is the numbers 0-9 and addition symbol '+') to a unique integer index. We also create a table mapping back the other way. 

To encode a string, we convert each character in the string to a one-hot vector with the character index as 1 and all other indices as 0. We also feed in the parameter `num_rows` as a maximum length of our string, equivalent to padding all strings to a specified length.

To decode an output array x, we take each vector of the string (representing a component) and decode it back into its most likely character. NOTE: I'm a little unclear about how we deal with variable-length answers. Maybe those are just decoded into space characters?

In [2]:
class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one hot integer representation
    + Decode the one hot integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        """Initialize character table.
        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One hot encode given string C.
        # Arguments
            num_rows: Number of rows in the returned one hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)


In [3]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'


## Model Parameters

These parameters are now heuristically chosen based on the data in our database.

MAXLEN is the maximum length of any one input (or can be truncated below that). Chosen as 14, for max input length from UCLA dataset.  
TRAINING_SIZE is an approximation (I think) for the number of examples in our dataset.  
`chars` is a string with all characters that can be used in our inputs/outputs. We also add a space for padding.

REVERSE is not currently used, but it should indicate whether we feed the inputs in standard order or reversed.

In [4]:
# Parameters for the model and dataset.
TRAINING_SIZE = 50000
MAXLEN_INPUT = 14
MAXLEN_OUTPUT = 15
REVERSE = True

chars = 'abcdefghijklmnopqrstuvwxyz '
ctable = CharacterTable(chars)


## Data Processing for Verb Tenses

We read in the data from our CSV of UCLA's verb tenses. Then we vectorize them using the Character class and model parameters defined above.

In [5]:
data_file = 'wordlist.csv'

import re # For stripping non-alphanumeric characters
present = []
past = []
with open(data_file) as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
        if not (re.search(r'\W+', row[0]) or re.search(r'\W+', row[1])):
            # Only add if all alphanumeric
            # Then pad inputs with spaces
            present_word = row[0] + ' ' * (MAXLEN_INPUT - len(row[0]))
            if REVERSE:
                # Reverse the query, e.g., '12+345  ' becomes '  543+21'.
                present_word = present_word[::-1]
            present.append(present_word)
            past_word = row[1] + ' ' * (MAXLEN_OUTPUT - len(row[1]))
            past.append(past_word)

print(list(zip(present, past))[:10])
print("MAXLEN of input: ", max([len(s) for s in present]))
print("MAXLEN of output: ", max([len(s) for s in past]))

assert(len(present)==len(past))
print("Num examples: ", len(present))

[('       nodnaba', 'abandoned      '), ('         esaba', 'abased         '), ('         hsaba', 'abashed        '), ('         etaba', 'abated         '), ('    etaiverbba', 'abbreviated    '), ('      etacidba', 'abdicated      '), ('        tcudba', 'abducted       '), ('          teba', 'abetted        '), ('         rohba', 'abhorred       '), ('         ediba', 'abode          ')]
MAXLEN of input:  16
MAXLEN of output:  17
Num examples:  6872


## Data Processing

Vectorize data using the Character library and previously set parameters

In [82]:
print('Vectorization...')
x = np.zeros((len(past), MAXLEN_INPUT, len(chars)), dtype=np.bool)
y = np.zeros((len(past), MAXLEN_OUTPUT, len(chars)), dtype=np.bool)
for i, word in enumerate(present):
    x[i] = ctable.encode(word, MAXLEN_INPUT)
for i, word in enumerate(past):
    y[i] = ctable.encode(word, MAXLEN_OUTPUT)

# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

Vectorization...
Training Data:
(1930, 14, 27)
(1930, 15, 27)
Validation Data:
(214, 14, 27)
(214, 15, 27)


## Setting up Model

In [89]:
# Try replacing GRU, or SimpleRNN.
RNN = layers.LSTM
HIDDEN_SIZE = 512
BATCH_SIZE = 128
LAYERS = 3


print('Build model...')
model = Sequential()
# "Encode" the input sequence using an RNN, producing an output of HIDDEN_SIZE.
# Note: In a situation where your input sequences have a variable length,
# use input_shape=(None, num_feature).
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN_INPUT, len(chars))))
# As the decoder RNN's input, repeatedly provide with the last hidden state of
# RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
# length of output, e.g., when DIGITS=3, max output is 999+999=1998.
model.add(layers.RepeatVector(MAXLEN_OUTPUT))
# The decoder RNN could be multiple layers stacked or a single layer.
for _ in range(LAYERS):
    # By setting return_sequences to True, return not only the last output but
    # all the outputs so far in the form of (num_samples, timesteps,
    # output_dim). This is necessary as TimeDistributed in the below expects
    # the first dimension to be the timesteps.
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

# Apply a dense layer to the every temporal slice of an input. For each of step
# of the output sequence, decide which character should be chosen.
model.add(layers.TimeDistributed(layers.Dense(len(chars))))
model.add(layers.Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_27 (LSTM)               (None, 512)               1105920   
_________________________________________________________________
repeat_vector_11 (RepeatVect (None, 15, 512)           0         
_________________________________________________________________
lstm_28 (LSTM)               (None, 15, 512)           2099200   
_________________________________________________________________
lstm_29 (LSTM)               (None, 15, 512)           2099200   
_________________________________________________________________
lstm_30 (LSTM)               (None, 15, 512)           2099200   
_________________________________________________________________
time_distributed_11 (TimeDis (None, 15, 27)            13851     
_________________________________________________________________
activation_11 (Activation)   (None, 15, 27)            0     

## Training, Validating Model

In [90]:
# Print validation examples and classifications from the dataset
def classify_val_examples():
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)

#Takes list of past loss and val_loss values, plots them
def plot_learning_curves(loss, val_loss):
    pyplot.clf()
    pyplot.plot(loss)
    pyplot.plot(val_loss)
    pyplot.title('model train vs validation loss')
    pyplot.ylabel('loss')
    pyplot.xlabel('epoch')
    pyplot.legend(['train', 'validation'], loc='upper right')
    pyplot.show()

In [ ]:
# Train the model each generation and show predictions against the validation
# dataset. Also graph the loss and validation loss.

loss = []
val_loss = []
for iteration in range(1, 500):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    history = model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    loss.append(history.history['loss'])
    val_loss.append(history.history['val_loss'])
    if iteration % 1 == 0:
        classify_val_examples()
    if iteration % 50 == 0:
        plot_learning_curves(loss, val_loss)
    
pyplot.title('model train vs validation loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'], loc='upper right')
pyplot.show()


--------------------------------------------------
Iteration 1
Train on 1930 samples, validate on 214 samples
Epoch 1/1
1930/1930 [==============================] - 20s 10ms/step - loss: 1.4069 - acc: 0.5840 - val_loss: 1.4501 - val_acc: 0.5617
Q please         T pleased         ☒ eeeeeed        
Q ponder         T pondered        ☒ eeeeeed        
Q coordinate     T coordinated     ☒ eeeeeeeeed     
Q reward         T rewarded        ☒ eeeeeed        
Q worry          T worried         ☒ eeeeee         
Q complement     T complemented    ☒ eeeeeeeeed     
Q foster         T fostered        ☒ eeeeeed        
Q bay            T bayed           ☒ eeeee          
Q core           T cored           ☒ eeeeed         
Q weigh          T weighed         ☒ eeeeed         

--------------------------------------------------
Iteration 2
Train on 1930 samples, validate on 214 samples
Epoch 1/1
1930/1930 [==============================] - 21s 11ms/step - loss: 1.3701 - acc: 0.5929 - val_loss: 1.6

## Assessing Underfitting, Overfitting

Here we plot the model's results to diagnose underfitting or overfitting.

We'll plot the learning curves with training loss and validation loss to understand what happens. Machine Learning Yearning might help us understand this too.

In [66]:
%matplotlib inline

history = model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=600,
              validation_data=(x_val, y_val))
# plot train and validation loss
pyplot.plot(history.history['loss'])
pyplot.plot(history.history['val_loss'])
pyplot.title('model train vs validation loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'validation'], loc='upper right')
pyplot.show()

Train on 1930 samples, validate on 214 samples
Epoch 1/600
1930/1930 [==============================] - 3s 2ms/step - loss: 16.2367 - acc: 0.4288 - val_loss: 13.1607 - val_acc: 0.5012
Epoch 2/600
1930/1930 [==============================] - 1s 615us/step - loss: 11.3721 - acc: 0.4943 - val_loss: 9.2820 - val_acc: 0.5012
Epoch 3/600
1930/1930 [==============================] - 1s 660us/step - loss: 7.9315 - acc: 0.5038 - val_loss: 6.3251 - val_acc: 0.5274
Epoch 4/600
1930/1930 [==============================] - 1s 617us/step - loss: 5.3043 - acc: 0.5392 - val_loss: 4.1224 - val_acc: 0.5757
Epoch 5/600
1930/1930 [==============================] - 1s 667us/step - loss: 3.4177 - acc: 0.5678 - val_loss: 2.6190 - val_acc: 0.5732
Epoch 6/600
1930/1930 [==============================] - 1s 621us/step - loss: 2.2065 - acc: 0.5794 - val_loss: 1.7840 - val_acc: 0.5819
Epoch 7/600
1930/1930 [==============================] - 1s 609us/step - loss: 1.6438 - acc: 0.5798 - val_loss: 1.5287 - val_acc: 

1930/1930 [==============================] - 1s 589us/step - loss: 1.2479 - acc: 0.6424 - val_loss: 1.3026 - val_acc: 0.6059
Epoch 61/600
1930/1930 [==============================] - 1s 658us/step - loss: 1.2657 - acc: 0.6327 - val_loss: 1.2475 - val_acc: 0.6293
Epoch 62/600
1930/1930 [==============================] - 1s 682us/step - loss: 1.2452 - acc: 0.6430 - val_loss: 1.2669 - val_acc: 0.6274
Epoch 63/600
1930/1930 [==============================] - 1s 713us/step - loss: 1.2403 - acc: 0.6461 - val_loss: 1.2296 - val_acc: 0.6436
Epoch 64/600
1930/1930 [==============================] - 1s 713us/step - loss: 1.2300 - acc: 0.6527 - val_loss: 1.2156 - val_acc: 0.6592
Epoch 65/600
1930/1930 [==============================] - 1s 703us/step - loss: 1.2233 - acc: 0.6529 - val_loss: 1.2466 - val_acc: 0.6308
Epoch 66/600
1930/1930 [==============================] - 1s 687us/step - loss: 1.2351 - acc: 0.6473 - val_loss: 1.2176 - val_acc: 0.6498
Epoch 67/600
1930/1930 [=======================

1930/1930 [==============================] - 1s 575us/step - loss: 1.0149 - acc: 0.7060 - val_loss: 1.0364 - val_acc: 0.7016
Epoch 179/600
1930/1930 [==============================] - 1s 586us/step - loss: 1.0168 - acc: 0.7062 - val_loss: 1.0164 - val_acc: 0.7047
Epoch 180/600
1930/1930 [==============================] - 1s 587us/step - loss: 1.0034 - acc: 0.7087 - val_loss: 1.0323 - val_acc: 0.7016
Epoch 181/600
1930/1930 [==============================] - 1s 586us/step - loss: 1.0318 - acc: 0.7014 - val_loss: 1.1364 - val_acc: 0.6798
Epoch 182/600
1930/1930 [==============================] - 1s 575us/step - loss: 1.1142 - acc: 0.6820 - val_loss: 1.0871 - val_acc: 0.6916
Epoch 183/600
1930/1930 [==============================] - 1s 570us/step - loss: 1.0476 - acc: 0.6989 - val_loss: 1.0341 - val_acc: 0.7022
Epoch 184/600
1930/1930 [==============================] - 1s 570us/step - loss: 1.0126 - acc: 0.7062 - val_loss: 1.0250 - val_acc: 0.7019
Epoch 185/600
1930/1930 [================

1930/1930 [==============================] - 1s 589us/step - loss: 0.9506 - acc: 0.7244 - val_loss: 0.9825 - val_acc: 0.7156
Epoch 238/600
1930/1930 [==============================] - 1s 573us/step - loss: 0.9435 - acc: 0.7268 - val_loss: 0.9664 - val_acc: 0.7206
Epoch 239/600
1930/1930 [==============================] - 1s 575us/step - loss: 0.9297 - acc: 0.7297 - val_loss: 0.9660 - val_acc: 0.7206
Epoch 240/600
1930/1930 [==============================] - 1s 573us/step - loss: 0.9312 - acc: 0.7285 - val_loss: 0.9967 - val_acc: 0.7115
Epoch 241/600
1930/1930 [==============================] - 1s 589us/step - loss: 0.9388 - acc: 0.7288 - val_loss: 0.9724 - val_acc: 0.7206
Epoch 242/600
1930/1930 [==============================] - 1s 595us/step - loss: 0.9280 - acc: 0.7304 - val_loss: 0.9679 - val_acc: 0.7218
Epoch 243/600
1930/1930 [==============================] - 1s 576us/step - loss: 0.9231 - acc: 0.7318 - val_loss: 0.9621 - val_acc: 0.7237
Epoch 244/600
1930/1930 [================

1930/1930 [==============================] - 1s 572us/step - loss: 0.8467 - acc: 0.7583 - val_loss: 0.9201 - val_acc: 0.7364
Epoch 297/600
1930/1930 [==============================] - 1s 570us/step - loss: 0.8371 - acc: 0.7613 - val_loss: 0.9301 - val_acc: 0.7368
Epoch 298/600
1930/1930 [==============================] - 1s 576us/step - loss: 0.8330 - acc: 0.7630 - val_loss: 0.9332 - val_acc: 0.7312
Epoch 299/600
1930/1930 [==============================] - 1s 575us/step - loss: 0.8352 - acc: 0.7637 - val_loss: 0.9068 - val_acc: 0.7439
Epoch 300/600
1930/1930 [==============================] - 1s 589us/step - loss: 0.8238 - acc: 0.7656 - val_loss: 0.9358 - val_acc: 0.7346
Epoch 301/600
1930/1930 [==============================] - 1s 574us/step - loss: 0.8528 - acc: 0.7554 - val_loss: 0.9123 - val_acc: 0.7389
Epoch 302/600
1930/1930 [==============================] - 1s 571us/step - loss: 0.8414 - acc: 0.7589 - val_loss: 0.9292 - val_acc: 0.7389
Epoch 303/600
1930/1930 [================

1930/1930 [==============================] - 1s 667us/step - loss: 0.7701 - acc: 0.7845 - val_loss: 0.8956 - val_acc: 0.7517
Epoch 356/600
1930/1930 [==============================] - 1s 662us/step - loss: 0.7391 - acc: 0.7935 - val_loss: 0.8583 - val_acc: 0.7611
Epoch 357/600
1930/1930 [==============================] - 1s 689us/step - loss: 0.7336 - acc: 0.7937 - val_loss: 0.8610 - val_acc: 0.7607
Epoch 358/600
1930/1930 [==============================] - 1s 624us/step - loss: 0.7189 - acc: 0.8004 - val_loss: 0.8520 - val_acc: 0.7626
Epoch 359/600
1930/1930 [==============================] - 1s 639us/step - loss: 0.7128 - acc: 0.8014 - val_loss: 0.8641 - val_acc: 0.7570
Epoch 360/600
1930/1930 [==============================] - 1s 711us/step - loss: 0.7169 - acc: 0.8000 - val_loss: 0.8660 - val_acc: 0.7579
Epoch 361/600
1930/1930 [==============================] - 1s 710us/step - loss: 0.7030 - acc: 0.8044 - val_loss: 0.8365 - val_acc: 0.7673
Epoch 362/600
1930/1930 [================

1930/1930 [==============================] - 1s 613us/step - loss: 0.6815 - acc: 0.8117 - val_loss: 0.9299 - val_acc: 0.7474
Epoch 415/600
1930/1930 [==============================] - 1s 610us/step - loss: 0.7463 - acc: 0.7960 - val_loss: 0.8846 - val_acc: 0.7601
Epoch 416/600
1930/1930 [==============================] - 1s 623us/step - loss: 0.6482 - acc: 0.8249 - val_loss: 0.8379 - val_acc: 0.7692
Epoch 417/600
1930/1930 [==============================] - 1s 675us/step - loss: 0.6199 - acc: 0.8314 - val_loss: 0.8229 - val_acc: 0.7757
Epoch 418/600
1930/1930 [==============================] - 1s 618us/step - loss: 0.5880 - acc: 0.8393 - val_loss: 0.8267 - val_acc: 0.7701
Epoch 419/600
1930/1930 [==============================] - 1s 622us/step - loss: 0.5880 - acc: 0.8398 - val_loss: 0.8334 - val_acc: 0.7692
Epoch 420/600
1930/1930 [==============================] - 1s 640us/step - loss: 0.5874 - acc: 0.8393 - val_loss: 0.8284 - val_acc: 0.7685
Epoch 421/600
1930/1930 [================

KeyboardInterrupt: 